In [1]:
from util import *
from calc import *
from dask.distributed import wait

client=get_rmm_client()

root_path='/home/yuninze/res'
data_name='uiop.zarr'

storage=init_zarr(root_path=root_path, data_name=data_name)

In [2]:
expr=to_da(storage.expr)
expr

dask.array<array, shape=(120, 37637567), dtype=int8, chunksize=(20, 5880870), chunktype=numpy.ndarray>

In [3]:
expr_name=storage['name'].__array__()
expr_name.shape

(37637567,)

In [4]:
def _parse_expr_name(x):
  return x[:x.find('_')]

parse_expr_name=np.vectorize(_parse_expr_name)
expr_name_clean=parse_expr_name(expr_name)

In [5]:
expr_name_target=pd.read_csv(
  '/home/yuninze/res/work/snpid-cdrsobz_lingress-0.001.csv'
).SNPID.to_numpy(dtype='U23')
expr_name_target

array(['10:2242119:TACAC:T', '10:2992214:G:A', '10:2992874:T:C', ...,
       '9:138317064:G:A', '9:138317066:C:T', '9:140383174:C:A'],
      dtype='<U23')

In [6]:
expr_name_clean=to_da(expr_name_clean)
expr_name_target=to_da(expr_name_target)

In [7]:
expr_name_clean_exist=da.isin(expr_name_clean,expr_name_target,assume_unique=False).compute()

/home/yuninze/conda/envs/origin/lib/python3.10/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 3.23 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [8]:
expr_df=pd.DataFrame(
  data=expr[:, expr_name_clean_exist].compute(),
  index=storage.ind.__array__(),
  columns=expr_name_clean[expr_name_clean_exist].compute()
)

/home/yuninze/conda/envs/origin/lib/python3.10/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 3.22 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [9]:
if expr_df.shape[0] == 120:
  expr_df.to_csv(os.path.join(root_path,'work','expr-sign-cdrsobz.csv'))